In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import qm
import pandas as pd
import numpy as np
import networkx as nx
import concurrent.futures
import par_helper as ph
import random
import robustness as rb
import pickle

In [3]:
def gen_random_configuration_network_canalizing(din, dout, bias=0.5):
    '''
    din: list of in-degree
    dout: list of out-degree
    bias: threshold for random Boolean logic
    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.DiGraph(nx.directed_configuration_model(din, dout))
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
                    
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            canalizing_variable = random.randrange(len(inf))
            canalizing_value = int(random.random() < 0.5)
            canalized_value = int(random.random() < 0.5)
            for j in bool_states:
                if j[canalizing_variable] == str(canalizing_value):
                    g.nodes[n]['update_rules'][j] = canalized_value
                else: 
                    g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [1]:
networkModel =[ 'bortezomib',
                # 'igvh',
                'apoptosis',
                # 'aurora',
                #'bt474_long',
                'bt474_short',
                # 'cd4t',
                'colitis',
                'death',
                # 'egfr',
                # 'erbb',
                # 'fa_brca',
                # 'fa_check',
                #'hcc1954_long',
                'hcc1954_short',
                'hgf',
                'mammalian',
                # 'mammalian_2006',
                'mapk',
                'oxidative',
                # 'pro_inflammatory',
                'fibroblasts',
                #'skbr3_long',
                'skbr3_short',
                'tlgl_2008',
                'tlgl_2011',
                # 'tlgl_2011_reduced',
                # 'prostate',
                'migration']

In [4]:
for Model in networkModel:
    Prefix, Suffix = 'n', 'n'
    TEMP = jR.cellcollective(Model, Prefix, Suffix, directory='../')

    BooleanRuleFileName = TEMP['BooleanRule_filename']
    network_name = TEMP['network_name']

    NumInputs = TEMP['num_inputs']
    NumInputConditions = TEMP['num_input_conditions']

    InputConditions = TEMP['input_conditions']

    OutputNodes = TEMP['output_nodes']
    InputNodes = TEMP['input_nodes']
    
    Mapping = TEMP['mapping']
    InverseMapping = TEMP['inverse_mapping']
    GRead = TEMP['Gread']
    ReadNodes = TEMP['read_nodes']
    
    GCopy = GRead.copy()
    GCopy.remove_edges_from(nx.selfloop_edges(GCopy))
    Din = list(d for n, d in GCopy.in_degree())
    Dout = list(d for n, d in GCopy.out_degree())
    
#     result_pd = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 
#                                   'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 
#                                   'redundancy'])
#     for idx in range(0, 500):
#         temp = r3_random_configuration_network_canalizing(GRead, bias=0.5, prefix='n', suffix='n', num_worker=10, sampling_num=10)
#         temp['network_idx'] = idx
#         result_pd = result_pd.append(temp, ignore_index=True)

    with concurrent.futures.ProcessPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(gen_random_configuration_network_canalizing, din=Din, dout=Dout, bias=0.5): network_idx for network_idx in range(500)}

    for future in concurrent.futures.as_completed(futures):
        network_idx = futures[future]
        temp, _ = future.result()
#         temp['network_idx'] = network_idx
# configuration_models
        with open('../networks/configuration_models/'+Model+'_' + str(network_idx) + '.pck', 'wb') as f:
            pickle.dump(temp, f)
#     result_pd.to_csv('../data/'+ Model + '_canalizing.csv')

In [10]:
# def r3_test(g, bias=0.5, prefix='n', suffix='n', num_worker=5, sampling_num=10):
    
#     read_nodes = list(g.nodes)
#     mapping = {}  # nodename to number index
#     inverse_mapping = {}  # number index to nodename
#     read_nodes_dict = {}
#     inverse_read_nodes_dict = {}
#     for i, node in enumerate(read_nodes):
#         index = prefix + str(i) + suffix
#         mapping[str(node)] = index
#         inverse_mapping[index] = str(node)
#         mapping['~' + str(node)] = '~' + index
#         inverse_mapping['~' + index] = '~' + str(node)
#         read_nodes_dict[i] = str(node)
#         inverse_read_nodes_dict[str(node)] = i
    
#     input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
#     output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
#     num_inputs = len(input_nodes)
#     num_input_conditions = 2 ** num_inputs
#     input_conditions = np.ndarray((num_inputs, 2), dtype=object)
#     for idx, input_node in enumerate(input_nodes):
#         input_conditions[idx, 0] = '~' + str(input_node)
#         input_conditions[idx, 1] = str(input_node)
    
    
#     output_nodes_ex = []
#     output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
#     output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
#     input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
#     GExpanded = ph.par_get_expanded_network(g, prefix=prefix, suffix=suffix, worker=num_worker)
# #     GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)
#     TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
#                                             constant_nodes=[])
#     LDOIs = TempGIOW['LDOIs']
#     GeneLDOIs = TempGIOW['gene_LDOIs']
#     Conflicts = TempGIOW['conflicts']
#     GeneConflicts = TempGIOW['gene_conflicts']
#     IORelation = TempGIOW['io_relation']
#     GRemained = TempGIOW['G_remained']

#     R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
#     R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
# #     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
# #     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
#     RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
#                                      IORelation,
#                                      R0['ineffective_mutations'], R1['r1_mutations'])
    
#     NodeList = set(read_nodes_dict.values())
#     NodeList.difference_update(input_nodes)
#     NodeList.difference_update(output_nodes)
#     C0, C1, C2, C3 = [], [], [], []
#     for NODE in NodeList:
#         negNODE = '~' + NODE
#         if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
# #             nodeClass = 'C0'
#             C0.append(NODE)
#         elif NODE in RN['rn_mutations']:
#             if RN['rn_mutations'][NODE] == 'R1':
# #                 nodeClass = 'C1'
#                 C1.append(NODE)
# #                 canalizing = IIDC['iid_canalizing'][NODE]
# #                 unreachable = UN['input_unreachable'][NODE]
#             else:
# #                 nodeClass = 'C2'
#                 C2.append(NODE)
# #                 canalizing = IIDC['iid_canalizing'][NODE]
# #                 unreachable = UN['input_unreachable'][NODE]
#         elif negNODE in RN['rn_mutations']:
#             if RN['rn_mutations'][negNODE] == 'R1':
# #                 nodeClass = 'C1'
#                 C1.append(NODE)
# #                 canalizing = IIDC['iid_canalizing'][NODE]
# #                 unreachable = UN['input_unreachable'][NODE]
#         else:
# #             nodeClass = 'C3'
#             C3.append(NODE)
# #             canalizing = IIDC['iid_canalizing'][NODE]
# #             unreachable = UN['input_unreachable'][NODE]
    

#     robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
#     robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
#     robustness = []
#     for key, val in robustness_pa.items():
#         robustness.append(val['oe'])
#         robustness.append(val['ko'])
# #     print(np.mean(robustness))
    
    
#     # deterministic_IO
#     empty_io = [x for x in IORelation[0,:] if len(x) == 0]
#     len_io = [len(x) for x in IORelation[0,:]]
#     empty_io_ratio = len(empty_io) / num_input_conditions
#     deterministic_io_ratio = sum(len_io) / (num_input_conditions * len(output_nodes))
    
    
#     LDOI_union = set()
#     for LDOI in LDOIs[0,:]:
#         LDOI_union = LDOI_union.union(LDOI)
#     LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
#     GSub = nx.subgraph(GExpanded, LDOI_union)
#     io_count = 0
#     io_pathway_count = 0
#     for i in range(num_input_conditions):
#         # G_copied = G_expanded.copy()
#         BinaryBit = np.binary_repr(i, width=num_inputs)
#         InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
#         # source_nodes = input_condition + constant_nodes
#         Sources = set([x for x in InputCondition])
#         # for node in source:
#         #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
#         #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

#         Targets = set([x for x in IORelation[0, i]])
#         if len(Targets) > 0:
#             for s in Sources:
#                 for t in Targets:
#                     if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
#                         io_count += 1
#                         len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
#                         for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
#                             io_pathway_count += 1

#     if io_count == 0:
#         redundancy = -1
#     else:
#         redundancy = io_pathway_count/float(io_count)
    
#     return {'size_of_network': len(g), 'num_link': len(g.edges), 'num_input': len(input_nodes), 'num_output': len(output_nodes),
#             'C0': len(C0), 'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 
#             'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip,
#             'empty_io_ratio': empty_io_ratio, 'deterministic_io_ratio':deterministic_io_ratio,
#             'redundancy': redundancy}